# Prioritize Delay-Sensitive Traffic Using QOS
###### <sup>Use case: Paban Sarma (pasarma@cisco.com); Developed by: Sarah Samuel (sasamuel@cisco.com). </sup>

Quality of Service (QoS) is the technique of prioritizing traffic flows and providing preferential forwarding for higher-priority packets. The fundamental reason for implementing QoS in your network is to provide better service for certain traffic flows.

There are 3 ways to achieve this on a Service-Provider (SP) network using QoS:
* Uniform Mode QoS
* Pipe Mode QoS
* Short-pipe Mode QoS

This notebook demonstrates uniform mode QoS, whereby the packets of delay-sensitive traffic is routed with high-priority, across the SP network.

### NETWORK TOPOLOGY
This notebook brings up a Service Provider network topology, with 2 Provider-Edge (PE) routers and one Provider (P) router. After the base configuration of OSPF and MPLS is up, QoS is applied on the ingress and egress interfaces of PE and P routers in the network to ensure that delay sensitive traffic such as voice and video are given high priroity.

There are 2 traffic generator ports in this network, which represent 2 Customer Edge (CE) routers. Along with that, there are 2 other CE routers present. Traffic flows from one traffic generator port to the other to simulate customer traffic across the SP network. Ping traffic is also present from CE1 to CE2.

![Topology](tgn-ospf-mpls.png)

In order to ensure higher priority is provided for video traffic, the customer marks the video traffic packets with certain values in the packet header, in this case the DSCP bits in the IP header are marked for expedited forwarding (ef). The service-provider is informed of this marking so that they can easily give preferential treatment to the marked packets.

In the ingress interface of the SP routers, the traffic packets are classified based on the values marked in the header of the incoming packet and the video traffic is given the highest priority over the other forms of traffic.

## Configuration Steps
* [Bring up emulator session with the base network and configurations](#step1)
* [Verify the base network configurations](#step2)
* [Apply the QoS configurations for PE1](#step5)
* [Apply the QoS configurations on other routers in the MPLS network](#step11)
* [Start Traffic from TREX Traffic Generator](#step100)
* [Verify the QoS configurations](#step7)


## <a name="step1"></a>Bring up emulator session with the base network and configurations

`The below cell installs a couple of python packages essential for running the traffic generator. You have to execute the cell below by clicking the play button.`

In [1]:
!pip install paramiko
!pip install paramiko-expect

     |████████████████████████████████| 206 kB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 5.9 MB/s  eta 0:00:01
     |████████████████████████████████| 961 kB 10.9 MB/s eta 0:00:01


`In the cell below, the python module SP_tgn contains the code to set up the python environment, the simulated 5-router topology and the linux server according to the network topology diagram, above. The sim.start() API brings up the emulator as per the network topology. Refer the SP_tgn.py file in this repository to have a look at the python module.`

In [2]:
from SP_tgn import *

sim = Vxr()
sim.no_image_copy=True
sim.clean()
print("Sim clean: Done")
print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")

try:
    sim.start(cfg)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

INFO:pyvxr.vxr:v1.1.0 2021-04-27 09:30 output_dir:vxr.out
INFO:pyvxr.vxr:bd8e02377cc7:/home/vxr/notebooks/Put-Technology-to-Work/QoS
INFO:pyvxr.vxr:Extracting vxr version from '/opt/cisco/vxr2/latest/setup.sh' file.
INFO:pyvxr.vxr_session:Starting a local bash session for user:vxr


3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]
Sim clean: Done
Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.


INFO:pyvxr.sim:Launch: sim_dir:/nobackup/vxr/pyvxr/lrrf79dilr sim_rel:/opt/cisco/vxr2/latest
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)
INFO:pyvxr.sim:Starting vxr: 'sim --skiphomecheck -n '
INFO:pyvxr.sim:Vxr up on host localhost
INFO:pyvxr.vxr:Getting port vector files for:rce1, rce2, rp1, rpe1, rpe2, trex
INFO:pyvxr.console:rpe1:wait for XR login prompt (console output captured in vxr.out/logs/console.rpe1.log)
INFO:pyvxr.console:rce2:wait for XR login prompt (console output captured in vxr.out/logs/console.rce2.log)
INFO:pyvxr.console:rce1:wait for XR login prompt (console output captured in vxr.out/logs/console.rce1.log)
INFO:pyvxr.console:rpe2:wait for XR login prompt (console output captured in vxr.out/logs/console.rpe2.log)
INFO:pyvxr.console:rp1:wait for XR login prompt (console output captured in vxr.out/logs/console.rp1.log)
INFO:pyvxr.console:rp1:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.c

Sim status:  {'localhost': 'running'}


`Play the cell below to obtain the telnet and ssh IP-addresses and ports to access the devices.`

In [3]:
print('Consoles can be reached by:')
print('P1: ', get_telnet_cmd(sim, 'rp1'), '\nP2: ', get_telnet_cmd(sim, 'rp2'), '\nPE1: ', get_telnet_cmd(sim, 'rpe1'), '\nPE2: ', get_telnet_cmd(sim, 'rpe2'), '\nTGN: ', get_telnet_cmd(sim, 'trex'))
print('or better:')
print('P1: ', get_ssh_cmd(sim, 'rp1'), '\nP2: ', get_ssh_cmd(sim, 'rp2'), '\nPE1: ', get_ssh_cmd(sim, 'rpe1'), '\nPE2: ', get_ssh_cmd(sim, 'rpe2'), '\nTGN: ', get_ssh_cmd(sim, 'trex', 'True'))
print('The password is cisco123')

Consoles can be reached by:
P1:  telnet 172.17.0.2 35212 
PE1:  telnet 172.17.0.2 34457 
PE2:  telnet 172.17.0.2 38917 
CE1:  telnet 172.17.0.2 35211 
CE2:  telnet 172.17.0.2 41832 
TGN:  telnet 172.17.0.2 36745
or better:
P1:  ssh cisco@172.17.0.2 -p61003 
PE1:  ssh cisco@172.17.0.2 -p62899 
PE2:  ssh cisco@172.17.0.2 -p63082 
CE1:  ssh cisco@172.17.0.2 -p63621 
CE2:  ssh cisco@172.17.0.2 -p60991 
TGN:  ssh root@172.17.0.2 -p62275
The password is cisco123


`The cell below accesses the telnet consoles of the routers so that we can apply the required QoS policies from this notebook.`

In [4]:
ports = sim.ports()
loginpe1 = telnetlib.Telnet(str(ports['rpe1']['HostAgent']) , str(ports['rpe1']['serial0']))
loginp1 = telnetlib.Telnet(str(ports['rp1']['HostAgent']) , str(ports['rp1']['serial0']))
loginp2 = telnetlib.Telnet(str(ports['rp2']['HostAgent']) , str(ports['rp2']['serial0']))
loginpe2 = telnetlib.Telnet(str(ports['rpe2']['HostAgent']) , str(ports['rpe2']['serial0']))
trexipaddress = str(ports['trex']['HostAgent'])
trexport = str(ports['trex']['xr_redir22'])

## <a name="step8"></a>Verify the base network configurations

`Ensure the basic reachability across the network by checking that the routing protocol (OSPF) and MPLS is up with the below commands`

In [10]:
print("Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.")
loginp1.write(('''
sh version
show ip ospf nei
show mpls ldp interface
show mpls ldp bindings brief
show run mpls ldp
show mpls ldp neigh br
show ip route
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',4)
print(line.decode())

Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.

br
show ip route
RP/0/RP0/CPU0:P1#sh version
Tue Apr 27 09:43:12.356 UTC
Cisco IOS XR Software, Version 7.3.1 LNT
Copyright (c) 2013-2021 by Cisco Systems, Inc.

Build Information:
 Built By     : ingunawa
 Built On     : Fri Feb 26 04:56:31 UTC 2021
 Build Host   : iox-ucs-020
 Workspace    : /auto/srcarchive17/prod/7.3.1/8000/ws
 Version      : 7.3.1
 Label        : 7.3.1

cisco 8000 (VXR)
cisco 8201 (VXR) processor with 32GB of memory
P1 uptime is 12 minutes
Cisco 8201 1RU Chassis

RP/0/RP0/CPU0:P1#show ip ospf nei
Tue Apr 27 09:43:12.469 UTC

* Indicates MADJ interface
# Indicates Neighbor awaiting BFD session up

Neighbors for OSPF 10

Neighbor ID     Pri   State           Dead Time   Address         Interface
209.165.200.229 1     FULL/DR         00:00:37    198.51.100.2    FourHundredGigE0/0/0/0
    Neighbor is up for 00:08:04
209.165.200.227 1     FULL/DR         00:00:34    192.0.2.2    

## <a name="step5"></a>Apply the QoS configurations on PE1

`The customer edge router has marked the DSCP bits as ef for the high priority video traffic. Play the below cell to configure the class-maps, policy-maps and apply the QoS configuration on the ingress interface of the PE1 router in order to the classify the traffic packets based on their DSCP value in the packet.`
* `The ingress policy-map PE-INGRESS ensures that the incoming packets marked with EF for the DSCP field is set to traffic class 7 and marks the mpls exp bits to 7`
* `The egress policy-map VIDEO-PRIORITY ensures that the packets with traffic class 7 are sent out through the high priority output queue of the egress interface. Hence these high-priority traffic packets will be forwarded with minimum delay`
![Topology](qos-pe.png)

In [9]:
print("Please wait. It may take 8-9 seconds to retrieve the information from the telnet console.")

loginpe1.write(('''
configure
class-map match-all DSCP-EF
match dscp ef
end-class-map
!
commit
root
!
policy-map PE-INGRESS
class DSCP-EF
set traffic-class 7
set mpls experimental imposition 7
!
class class-default
set traffic-class 0
!
end-policy-map
commit
root
!
interface FourHundredGigE0/0/0/0
service-policy input PE-INGRESS
commit
root
!
class-map match-any Video
match traffic-class 7
end-class-map
!
commit
root
policy-map VIDEO-PRIORITY
class Video
shape average 2 gbps
priority level 1
!
class class-default
!
end-policy-map
commit
root
!
interface FourHundredGigE0/0/0/1
service-policy output VIDEO-PRIORITY
commit
root
exit

''').encode('ascii'))
line = loginpe1.read_until(b'/r/n',8)
print(line.decode())

Please wait. It may take 8-9 seconds to retrieve the information from the telnet console.

RP/0/RP0/CPU0:PE1#configure
ce-policy input PE-INGRESS
commit
root
!
class-map match-any Video
match traffic-class 7
end-class-map
!
commit
root
policy-map VIDEO-PRIORITY
class Video
shape average 2 gbps
priority level 1
!
class class-default
!
end-policy-map
commit
root
!
interface FourHundredGigE0/0/0/1
service-policy output VIDEO-PRIORITY
commit
root
exit

Tue Apr 27 09:43:02.803 UTC
RP/0/RP0/CPU0:PE1(config)#class-map match-all DSCP-EF
RP/0/RP0/CPU0:PE1(config-cmap)#match dscp ef
RP/0/RP0/CPU0:PE1(config-cmap)#end-class-map
RP/0/RP0/CPU0:PE1(config)#!
RP/0/RP0/CPU0:PE1(config)#commit
Tue Apr 27 09:43:03.221 UTC
RP/0/RP0/CPU0:PE1(config)#root
RP/0/RP0/CPU0:PE1(config)#!
RP/0/RP0/CPU0:PE1(config)#policy-map PE-INGRESS
RP/0/RP0/CPU0:PE1(config-pmap)#class DSCP-EF
RP/0/RP0/CPU0:PE1(config-pmap-c)#set traffic-class 7
RP/0/RP0/CPU0:PE1(config-pmap-c)#set mpls experimental imposition 7
RP/0/RP0/CPU0

## <a name="step6"></a>Apply the QoS configurations on other routers in the MPLS network

`Play the below cell to configure the class-maps, policy-maps and apply the QoS configuration on the ingress interface of the P1 router in order to the classify the incoming traffic packets based on their MPLS EXP value. Then in the egress interface, ensure that these delay-sensitive traffic gets high priority and assured bandwidth.`
![Topology](qos-core.png)

In [11]:
print("Please wait. It may take 8-9 seconds to retrieve the information from the telnet console.")

loginp1.write(('''
configure
class-map match-any EXP7
match mpls experimental topmost 7 
end-class-map
commit
root
!
policy-map CORE-INGRESS
class EXP7
set traffic-class 7
set mpls experimental imposition 7 

!
class class-default
set traffic-class 0
!
end-policy-map
commit
root
!
class-map match-any Video
match traffic-class 7
end-class-map
!
commit
root
!
policy-map VIDEO-PRIORITY
class Video
shape average 2 gbps
priority level 1
!
class class-default
!
end-policy-map
commit
root
!
interface FourHundredGigE0/0/0/0
service-policy input CORE-INGRESS
commit
root
!
interface FourHundredGigE0/0/0/1
service-policy output VIDEO-PRIORITY
commit
root
exit

''').encode('ascii'))
line = loginp1.read_until(b'/r/n',8)
print(line.decode())

Please wait. It may take 8-9 seconds to retrieve the information from the telnet console.

ch-anyRP/0/RP0/CPU0:P1#configure
 EXP7
match mpls experimental topmost 7 
end-class-map
commit
root
!
policy-map CORE-INGRESS
class EXP7
set traffic-class 7
set mpls experimental imposition 7 

!
class class-default
set traffic-class 0
!
end-policy-map
commit
root
!
class-map match-any Video
match traffic-class 7
end-class-map
!
commit
root
!
policy-map VIDEO-PRIORITY
class Video
shape average 2 gbps
priority level 1
!
class class-default
!
end-policy-map
commit
root
!
interface FourHundredGigE0/0/0/0
service-policy output VIDEO-PRIORITY
commit
root
!
interface FourHundredGigE0/0/0/1
service-policy input CORE-INGRESS
commit
root
exit

Tue Apr 27 09:43:31.985 UTC
RP/0/RP0/CPU0:P1(config)#class-map match-any EXP7
RP/0/RP0/CPU0:P1(config-cmap)#match mpls experimental topmost 7 
RP/0/RP0/CPU0:P1(config-cmap)#end-class-map
RP/0/RP0/CPU0:P1(config)#commit
Tue Apr 27 09:43:32.383 UTC
RP/0/RP0/CPU0:P1(co

## <a name="step100"></a>Start traffic from TREX Traffic Generator

`The below cell is a call to a python function which logs into the Traffic Generator (TREX), performs the required configurations and then starts traffic across the SP network for an hour. Play this cell to start the traffic at 100pps.`
![traffic_flow](qos-traffic.png)

In [12]:
print("Please wait. Traffic Generator is starting....")

client1, client2, interact1, interact2 = generate_hipriority_traffic (trexipaddress, trexport)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Authentication (password) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 0] sftp session closed.


Last login: Tue Apr 27 09:33:17 2021
[root@localhost ~]# ifconfig eth1 10.0.0.1 netmask 255.255.255.0 up; ifconfig et h2 10.1.1.1 netmask 255.255.255.0 up
[root@localhost ~]# cd /opt/cisco/trex/latest/
[root@localhost latest]# ./t-rex-64 -i
Starting Scapy server....... Scapy server is started
Trying to bind to vfio-pci ...
Trying to compile and bind to igb_uio ...
ERROR: We don't have precompiled igb_uio.ko module for your kernel version.
Will try compiling automatically...
Success.

/usr/bin/python3 dpdk_nic_bind.py --bind=igb_uio 0000:00:04.0 0000:00:05.0 
The ports are bound/configured.
Starting  TRex v2.89 please wait  ... 
 set driver name net_virtio 
 driver capability  :
 set dpdk queues mode to ONE_QUE 
 Number of ports found: 2
zmq publisher at: tcp://*:4500
 wait 1 sec 

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Authentication (publickey) failed.
INFO:paramiko.transport:Authentication (password) successful!


Last login: Tue Apr 27 09:43:45 2021 from gateway
[root@localhost ~]# cd /opt/cisco/trex/latest/
[root@localhost latest]# ./trex-console
start -f stl/dscp_traffic1.py -d 1h -m 100pps -p 0

Using 'python3' as Python interpeter


Connecting to RPC server on localhost:4501                   [SUCCESS]


Connecting to publisher server on localhost:4500             [SUCCESS]


Acquiring ports [0, 1]:                                      [SUCCESS]


Server Info:

Server version:   v2.89 @ STL
Server mode:      Stateless
Server CPU:       1 x VXR
Ports count:      2 x 100.0Gbps @ Virtio network device	

-=TRex Console v3.0=-

Type 'help' or '?' for supported actions

trex>start -f stl/dscp_traffic1.py -d 1h -m 100pps -p 0


## <a name="step7"></a>Verify the QoS configurations using show commands
`Play the below cell to check the the outputs of the interface counters and the policy-map on the routers. The interface counters show the number of packets incoming or outgoing from the interface. And the policy-map counters show the number of packets for which the QoS is being applied. In the below show command outputs verify that the packet counters are incrementing in class DSCP-EF in the ingress policy-map and in class Video in the egress policy-map.`

In [14]:
print("Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.")

# Check the input and output policies applied on PE1 router.
loginpe1.write(('''
show interface FourHundredGigE0/0/0/0 accounting
show interface FourHundredGigE0/0/0/1 accounting
show policy-map interface FourHundredGigE0/0/0/0
show policy-map interface FourHundredGigE0/0/0/1
''').encode('ascii'))
line = loginpe1.read_until(b'/r/n',4)
print("***** LIVE OUTPUT FROM TELNET CONSOLE OF PE1 *****")
print(line.decode())

Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.
***** LIVE OUTPUT FROM TELNET CONSOLE OF PE1 *****

RP/0/RP0/CPU0:PE1#show interface FourHundredGigE0/0/0/2 accounting
how interface FourHundredGigE0/0/0/1 accounting
show policy-map interface FourHundredGigE0/0/0/2
show policy-map interface FourHundredGigE0/0/0/1
Tue Apr 27 09:44:55.260 UTC
FourHundredGigE0/0/0/2
  Protocol              Pkts In         Chars In     Pkts Out        Chars Out
  IPV4_UNICAST             5613                0            0                0
  ARP                         3              180            2               84


RP/0/RP0/CPU0:PE1#show interface FourHundredGigE0/0/0/1 accounting
Tue Apr 27 09:44:55.383 UTC
FourHundredGigE0/0/0/1
  Protocol              Pkts In         Chars In     Pkts Out        Chars Out
  IPV4_UNICAST               31                0            0                0
  MPLS                        0                0         5624            22496


`In the below show command outputs verify that the packet counters are incrementing in class EXP7 in the ingress policy-map and in class Video in the egress policy-map. This shows that even in subsequent routers in the MPLS network, delay-sensitive traffic can be given high priority.`

In [15]:
print("Please wait. It may take 4-5 seconds to retrieve the information from the telnet console.")

loginp1.write(('''
show interface FourHundredGigE0/0/0/0 accounting
show interface FourHundredGigE0/0/0/1 accounting
show policy-map interface FourHundredGigE0/0/0/0
show policy-map interface FourHundredGigE0/0/0/1
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',4)
print("***** LIVE OUTPUT FROM TELNET CONSOLE OF P1 *****")
print(line.decode())

***** LIVE OUTPUT FROM TELNET CONSOLE OF P1 *****

RP/0/RP0/CPU0:P1#show interface FourHundredGigE0/0/0/1 accounting
Tue Apr 27 09:45:05.191 UTC
FourHundredGigE0/0/0/1
  Protocol              Pkts In         Chars In     Pkts Out        Chars Out
  IPV4_UNICAST               33                0            0                0
  MPLS                     6548                0            0                0
  ARP                         2              148            2               84


RP/0/RP0/CPU0:P1#show interface FourHundredGigE0/0/0/0 accounting
Tue Apr 27 09:45:05.280 UTC
FourHundredGigE0/0/0/0
  Protocol              Pkts In         Chars In     Pkts Out        Chars Out
  IPV4_UNICAST               34                0         6556            26224
  ARP                         1               74            1               42


RP/0/RP0/CPU0:P1#show policy-map interface FourHundredGigE0/0/0/1
Tue Apr 27 09:45:05.359 UTC

FourHundredGigE0/0/0/1 input: CORE-INGRESS

Class EXP7
  Classi

`Close the telnet consoles and bring down the emulator after you are done with testing.`

In [16]:
loginpe1.close()
loginpe2.close()
logince1.close()
logince2.close()
loginp1.close()
client1.close()
client2.close()

In [17]:
sim.stop()

INFO:pyvxr.vxr:Stopping sim on host localhost (dir /nobackup/vxr/pyvxr/lrrf79dilr)
INFO:pyvxr.sim:Stopping previous simulation (if any)


In [18]:
sim.clean()
# Clean up sim scratch space
shutil.rmtree(sim_dir)

INFO:pyvxr.vxr:Cleaning sim on host localhost (dir /nobackup/vxr/pyvxr/lrrf79dilr)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)


Hope you now have a good idea on using QoS policies to ensure that high-priority traffic is forwarded through the network with minimum delay. If you have any comments or suggestions about this notebook, please reach out to sasamuel@cisco.com. We look forward to your feedback.